In [ ]:
from flask import Flask, request, jsonify, make_response, redirect, render_template
import urllib.request
# import urllib3.request
import cv2
from server.image_processing import ImageProcessing

app = Flask(__name__)
ip = ImageProcessing() 
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

user_agent = 'Mozilla/5.0 (iPhone; CPU iPhone OS 5_0 like Mac OS X) AppleWebKit/534.46'

@app.errorhandler(400)
def bad_request(e):
    # return also the code error
    return jsonify({"status": "not ok", "message": "this server could not understand your request"}), 400

@app.errorhandler(404)
def not_found(e):
    # return also the code error
    return jsonify({"status": "not found", "message": "route not found"}), 404

@app.errorhandler(500)
def not_found(e):
    # return also the code error
    return jsonify({"status": "internal error", "message": "internal error occurred in server"}), 500


@app.route('/detect', methods=['GET', 'POST'])
def detect_human_faces():
    if request.method == 'GET':
        if request.args.get('url'):
            with urllib.request.urlopen(request.args.get('url')) as url:
                image_with_boxes = ip.object_detection(url.read())
                retval, buffer = cv2.imencode('.jpg', image_with_boxes)
                response = make_response(buffer.tobytes())
                response.headers['Content-Type'] = 'image/jpeg'
                return response
                # return jsonify({"status": "ok", "result": ip.object_detection(url.read())}), 200
        else:
            return jsonify({"status": "bad request", "message": "Parameter url is not present"}), 400
    elif request.method == 'POST':
        # return render_template("index.html")
        if request.files.get("image"):
            return jsonify({"status": "ok", "result": ip.object_detection(request.files["image"].read())}), 200
        else:
            return jsonify({"status": "bad request", "message": "Parameter image is not present"}), 400
    else:
        return jsonify({"status": "failure", "message": "PUT method not supported for API"}), 405

@app.route('/', methods=["GET"])
def info_view():
    # List of routes for this API:
    output = {
        'info': 'GET /',
        'detect faces via POST': 'POST /detect',
        'detect faces via GET': 'GET /detect',
    }
    return jsonify(output), 200

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)


 * Running on http://localhost:9000/ (Press CTRL+C to quit)


Using device: cuda


127.0.0.1 - - [09/Feb/2020 09:35:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2020 09:36:15] "GET /detect?url=https://ovedma.ru/wp-content/uploads/2018/01/2kB1zcCyL1I-1.jpg HTTP/1.1" 200 -


In [1]:
from werkzeug.wrappers import Request, Response
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Feb/2020 09:33:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2020 09:33:51] "GET /favicon.ico HTTP/1.1" 404 -
